In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

## 0. 학습 준비
- 데이터셋 로드
- 환경 설정 (패키지 설치 및 로드)

In [2]:
import os

# === 입출력 경로 ===
os.environ.setdefault("CSV_PATH", "schedule_dataset_augmented_epoch_2.csv")   # seed 파일 경로

# 배치 단위 (한 번에 몇 개의 항목을 생성시킬지)
os.environ.setdefault("BATCH_SIZE", "10")

# 생성 다양성
os.environ.setdefault("TEMPERATURE", "0.8")
os.environ.setdefault("TOP_P", "0.9")

print("환경설정 완료")


환경설정 완료


In [3]:

# 환경 설정과 데이터 로드
import re, random, json, types
import numpy as np
import pandas as pd
from pathlib import Path

CSV_PATH = os.getenv("CSV_PATH")

df = pd.read_csv(CSV_PATH)
print("Loaded shape:", df.shape)
print(df.head())

Loaded shape: (2000, 8)
              sentence             domain    task  label  confidence   source  \
0     배경 러프 수정본 여기 있어요  design_production    NONE      0        0.99  teacher   
1  오늘 밤에 색감 체크 가능하신가요?  design_production  CREATE      1        0.92  teacher   
2        싱크표 다시 올려드릴게요  design_production    NONE      0        0.98  teacher   
3   금요일 오후 킥오프 미팅 할까요?  design_production  CREATE      1        0.95  teacher   
4       오늘 콘티 피드백만 주세요  design_production    NONE      0        0.97  teacher   

   seed_id  epoch  
0      0.0      0  
1      1.0      0  
2      2.0      0  
3      3.0      0  
4      4.0      0  


In [4]:
 
# 텍스트/라벨 컬럼 자동 감지 및 분할
from sklearn.model_selection import train_test_split

text_col = "sentence" if "sentence" in df.columns else df.columns[0]
label_col = "label" if "label" in df.columns else df.columns[-1]
print("Using columns -> text:", text_col, "label:", label_col)

df = df[[text_col, label_col]].dropna().reset_index(drop=True)
X = df[text_col].astype(str).values
y = df[label_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=38, stratify=y
)
len(X_train), len(X_test)

Using columns -> text: sentence label: label


(1600, 400)

## TF-IDF 분류기

In [9]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# 1) char ngram 기반
tfidf_char = Pipeline([
    ("tfidf", TfidfVectorizer(analyzer="char", ngram_range=(2,5), min_df=3)),
    ("clf", LinearSVC())
])
tfidf_char.fit(X_train, y_train)
pred_char = tfidf_char.predict(X_test)
acc_char = accuracy_score(y_test, pred_char)
print("TF‑IDF char Accuracy:", acc_char)
print(classification_report(y_test, pred_char))

# 2) word 기반
tfidf_word = Pipeline([
    ("tfidf", TfidfVectorizer(analyzer="word", token_pattern=r"(?u)\b\w+\b", min_df=3)),
    ("clf", LinearSVC())
])
tfidf_word.fit(X_train, y_train)
pred_word = tfidf_word.predict(X_test)
acc_word = accuracy_score(y_test, pred_word)
print("TF‑IDF word Accuracy:", acc_word)
print(classification_report(y_test, pred_word))

TF‑IDF char Accuracy: 0.9925
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       148
           1       0.99      1.00      0.99       252

    accuracy                           0.99       400
   macro avg       0.99      0.99      0.99       400
weighted avg       0.99      0.99      0.99       400

TF‑IDF word Accuracy: 0.9975
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       148
           1       1.00      1.00      1.00       252

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



In [10]:
import pandas as pd

df_labeled = pd.read_csv("labeled_sentences_voted.csv")
display(df_labeled.head())

,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
0,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 반갑습니다 상담사 #@이름#입니다,0,1.0,95,True
1,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 아까 제가 지금 상담을 하는 건데 들으세요,0,1.0,84,True
2,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,#@소속#에서 판매를 어떤 물건을 하든지 하게 되면은 판매 정책과 사업부가 있죠,0,1.0,87,True
3,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,상담하는 거는 그 사업을 위해서 상담하는 거고요,0,1.0,85,True
4,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그니까 저희는 유지관리를 상담해 드리는 상담실입니다,0,1.0,85,True


In [11]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

# X, y 설정
X_test = df_labeled["sentence"]
y_test = df_labeled["label_pred"]   # teacher 모델로 라벨링된 결과

# --- TF-IDF char 모델 ---
pred_char = tfidf_char.predict(X_test)
acc_char = accuracy_score(y_test, pred_char)
print("\n=== TF-IDF (char) 결과 ===")
print("Accuracy:", acc_char)
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_char))
print(classification_report(y_test, pred_char, digits=3))

# --- TF-IDF word 모델 ---
pred_word = tfidf_word.predict(X_test)
acc_word = accuracy_score(y_test, pred_word)
print("\n=== TF-IDF (word) 결과 ===")
print("Accuracy:", acc_word)
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_word))
print(classification_report(y_test, pred_word, digits=3))



=== TF-IDF (char) 결과 ===
Accuracy: 0.6827021494370522
Confusion Matrix:
 [[611 291]
 [ 19  56]]
              precision    recall  f1-score   support

           0      0.970     0.677     0.798       902
           1      0.161     0.747     0.265        75

    accuracy                          0.683       977
   macro avg      0.566     0.712     0.532       977
weighted avg      0.908     0.683     0.757       977


=== TF-IDF (word) 결과 ===
Accuracy: 0.34083930399181167
Confusion Matrix:
 [[272 630]
 [ 14  61]]
              precision    recall  f1-score   support

           0      0.951     0.302     0.458       902
           1      0.088     0.813     0.159        75

    accuracy                          0.341       977
   macro avg      0.520     0.557     0.309       977
weighted avg      0.885     0.341     0.435       977



In [12]:
import numpy as np
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, average_precision_score

def pick_thr_for_precision(y_true, scores, target_p=0.90):
    p, r, th = precision_recall_curve(y_true, scores)
    cands = [(pp, rr, tt) for pp, rr, tt in zip(p[:-1], r[:-1], th) if pp >= target_p]
    if cands:
        # precision 만족 중 recall 최대
        best = max(cands, key=lambda x: x[1])
        return best[2], {"P": best[0], "R": best[1], "picked": f"P≥{target_p}"}
    # 없으면 F1 최대 지점
    f1s = []
    for pp, rr, tt in zip(p[:-1], r[:-1], th):
        f1 = 0 if (pp+rr)==0 else 2*pp*rr/(pp+rr)
        f1s.append((f1, tt, pp, rr))
    f1s.sort(reverse=True, key=lambda x: x[0])
    f1, thr, pp, rr = f1s[0]
    return thr, {"P": pp, "R": rr, "F1": f1, "picked": "maxF1"}

def eval_with_thr(model, X, y, target_p=0.90, name="model"):
    scores = model.decision_function(X)  # LinearSVC 점수
    thr, info = pick_thr_for_precision(y, scores, target_p)
    y_pred = (scores >= thr).astype(int)
    P = precision_score(y, y_pred, zero_division=0)
    R = recall_score(y, y_pred, zero_division=0)
    F1 = f1_score(y, y_pred, zero_division=0)
    AP = average_precision_score(y, scores)
    print(f"[{name}] thr={thr:.4f} ({info['picked']}) | VAL-like P={info['P']:.3f}, R={info['R']:.3f}")
    print(f"TEST-ish  P={P:.3f} R={R:.3f} F1={F1:.3f} PR-AUC={AP:.3f}")
    return thr, (P,R,F1,AP)

# df_labeled 전체가 테스트셋이라면 그대로 적용 (혹은 내부에서 80/20 쪼개 임계값 고르고 동일 분포에 평가)
thr_char, _ = eval_with_thr(tfidf_char, X_test, y_test, target_p=0.90, name="TF-IDF char")
thr_word, _ = eval_with_thr(tfidf_word, X_test, y_test, target_p=0.90, name="TF-IDF word")


[TF-IDF char] thr=1.0867 (P≥0.9) | VAL-like P=1.000, R=0.013
TEST-ish  P=1.000 R=0.013 F1=0.026 PR-AUC=0.349
[TF-IDF word] thr=1.5366 (P≥0.9) | VAL-like P=1.000, R=0.013
TEST-ish  P=1.000 R=0.013 F1=0.026 PR-AUC=0.299


In [13]:
from sklearn.calibration import CalibratedClassifierCV
cal_char = CalibratedClassifierCV(tfidf_char, method="sigmoid", cv=3).fit(X_train, y_train)
probs = cal_char.predict_proba(X_test)[:,1]
# → precision 목표에 맞춰 threshold 선택

In [19]:
import numpy as np
from sklearn.metrics import (
    precision_recall_curve, precision_score, recall_score, f1_score,
    average_precision_score, confusion_matrix, classification_report
)

def pick_thr_for_precision(y_true, probs, target_p=0.90):
    p, r, th = precision_recall_curve(y_true, probs)
    # thresholds 길이는 p,r보다 1 작음
    cands = [(pp, rr, tt) for pp, rr, tt in zip(p[:-1], r[:-1], th) if pp >= target_p]
    if cands:
        # precision 만족 중 recall 최대 지점
        best = max(cands, key=lambda x: x[1])
        return best[2], {"picked_by": f"P≥{target_p}", "P": best[0], "R": best[1]}
    # fallback: F1 최대 지점
    f1s = []
    for pp, rr, tt in zip(p[:-1], r[:-1], th):
        f1 = 0 if (pp+rr)==0 else 2*pp*rr/(pp+rr)
        f1s.append((f1, tt, pp, rr))
    f1s.sort(reverse=True, key=lambda x: x[0])
    f1, thr, pp, rr = f1s[0]
    return thr, {"picked_by": "maxF1", "P": pp, "R": rr, "F1": f1}

# 1) 임계값 선택 (예: Precision 0.90 목표)
thr, info = pick_thr_for_precision(y_test, probs, target_p=0.9)
print(f"[Threshold] {info['picked_by']} -> thr={thr:.4f}  (VAL-like P={info['P']:.3f}, R={info['R']:.3f})")

# 2) 테스트셋 예측 및 지표
y_pred = (probs >= thr).astype(int)

P = precision_score(y_test, y_pred, zero_division=0)
R = recall_score(y_test, y_pred, zero_division=0)
F1 = f1_score(y_test, y_pred, zero_division=0)
AP = average_precision_score(y_test, probs)  # PR-AUC은 확률 기반
cm = confusion_matrix(y_test, y_pred, labels=[0,1])

print(f"[TEST] Precision={P:.3f}  Recall={R:.3f}  F1={F1:.3f}  PR-AUC={AP:.3f}")
print("Confusion Matrix [rows=true 0/1, cols=pred 0/1]:\n", cm)
print("Classification report:\n", classification_report(y_test, y_pred, digits=3))


[Threshold] P≥0.9 -> thr=0.9976  (VAL-like P=1.000, R=0.027)
[TEST] Precision=1.000  Recall=0.027  F1=0.052  PR-AUC=0.389
Confusion Matrix [rows=true 0/1, cols=pred 0/1]:
 [[902   0]
 [ 73   2]]
Classification report:
               precision    recall  f1-score   support

           0      0.925     1.000     0.961       902
           1      1.000     0.027     0.052        75

    accuracy                          0.925       977
   macro avg      0.963     0.513     0.507       977
weighted avg      0.931     0.925     0.891       977



In [5]:
 # 텍스트/라벨 컬럼 자동 감지 및 분할
from sklearn.model_selection import train_test_split

text_col = "sentence" if "sentence" in df.columns else df.columns[0]
label_col = "label" if "label" in df.columns else df.columns[-1]
print("Using columns -> text:", text_col, "label:", label_col)

df = df[[text_col, label_col]].dropna().reset_index(drop=True)
X_test = df[text_col].astype(str).values
Y_test = df[label_col].values

print(X_test, Y_test)

Using columns -> text: sentence label: label
['배경 러프 수정본 여기 있어요' '오늘 밤에 색감 체크 가능하신가요?' '싱크표 다시 올려드릴게요' ...
 '콘티 초안 좀 더 빨리 받을 수 있나요? 🙃' '금요일 리뷰 시간, 15분 늦춰도 괜찮아? 내일 아침에 다시 조율해볼게.'
 '오늘 콘티 피드백 좀 주실래요? 👀'] [0 1 0 ... 0 1 0]


In [8]:
# 3) 기존 데이터셋 평가

import joblib
from sklearn.metrics import accuracy_score, classification_report
model = joblib.load("artifacts/char_calibrated.joblib")

pred_char = model.predict(X_test)
acc_char = accuracy_score(Y_test, pred_char)
print("TF‑IDF char Accuracy:", acc_char)
print(classification_report(Y_test, pred_char))

TF‑IDF char Accuracy: 0.9985
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       740
           1       1.00      1.00      1.00      1260

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



## 모델 내보내기

In [27]:
import joblib, os

os.makedirs("artifacts", exist_ok=True)
joblib.dump(cal_char, "artifacts/char_calibrated.joblib")
print("✅ 모델 저장 완료:", os.path.abspath("artifacts/char_calibrated.joblib"))

✅ 모델 저장 완료: /home/j-k13s101/artifacts/char_calibrated.joblib
